In [1]:
!pip install tensorflow
!pip install opencv-python
!pip install numpy
!pip install matplotlib

In [7]:
import tensorflow as tf
import cv2
import numpy as np
from google.colab import files
from PIL import Image
import io
import os

model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False)

print("Upload USER image")
user_file = files.upload()

print("Upload PRODUCT image")
product_file = files.upload()

def read_uploaded_image(file_dict):
    key = list(file_dict.keys())[0]
    img_bytes = file_dict[key]
    image = Image.open(io.BytesIO(img_bytes)).convert('RGB')
    return cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

user_image = read_uploaded_image(user_file)
product_image = read_uploaded_image(product_file)

def try_on_model(user_image, product_image):
    try:
        user_image_resized = cv2.resize(user_image, (224, 224))
        product_image_resized = cv2.resize(product_image, (224, 224))

        user_input = np.expand_dims(user_image_resized, axis=0)
        product_input = np.expand_dims(product_image_resized, axis=0)

        user_features = model.predict(user_input)
        product_mask = model.predict(product_input)

        output_image = cv2.addWeighted(user_image_resized, 0.7, product_image_resized, 0.3, 0)
        return output_image

    except Exception as e:
        print("___________Error_______________",e)
        return None

output_image = try_on_model(user_image, product_image)
if output_image is not None:
    output_path = "output_image.jpg"
    cv2.imwrite(output_path, output_image)
    print(f"Output saved as {output_path}")
else:
    print("Failed to generate output image.")


/tmp/ipython-input-7-2376847875.py:9: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False)


Upload USER image


Saving Screenshot 2025-06-18 192924.png to Screenshot 2025-06-18 192924 (2).png
Upload PRODUCT image


Saving Screenshot 2025-06-18 193011.png to Screenshot 2025-06-18 193011 (3).png
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Output saved as output_image.jpg


In [ ]:
!pip install fastapi
!pip install uvicorn

In [ ]:
from fastapi import FastAPI
from fastapi.responses import FileResponse

app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "Try-on Feature API"}

@app.post("/try-on")
async def try_on(user_image: UploadFile = File(...), product_image: UploadFile = File(...)):
    user_image_bytes = await user_image.read()
    product_image_bytes = await product_image.read()

    user_image_np = np.frombuffer(user_image_bytes, np.uint8)
    product_image_np = np.frombuffer(product_image_bytes, np.uint8)

    user_image_cv = cv2.imdecode(user_image_np, cv2.IMREAD_COLOR)
    product_image_cv = cv2.imdecode(product_image_np, cv2.IMREAD_COLOR)

    output_image = try_on_model(user_image_cv, product_image_cv)

    cv2.imwrite('output_image.jpg', output_image)

    return FileResponse('output_image.jpg')

In [ ]:
!uvicorn app:app --reload